In [1]:
# Create an interactive dashboard
import numpy as np
import pandas as pd
import json
import os
from tqdm import tqdm
import pickle
from transformers import AutoTokenizer, AutoModel
import torch
import altair as alt
from dash import Dash, Input, Output, callback, dcc, html, dash_table, State
import dash
from jupyter_dash import JupyterDash
from vega_datasets import data
import dash_vega_components as dvc

# Example

In [2]:
# Passing a stylesheet is not required
app = Dash(
    __name__,
    # external_stylesheets=["https://codepen.io/chriddyp/pen/bWLwgP.css"]
)

app.layout = html.Div(
    [
        html.H1("Altair Chart"),
        dcc.Dropdown(["All", "USA", "Europe", "Japan"], "All", id="origin-dropdown"),
        # Optionally, you can pass options to the Vega component.
        # See https://github.com/vega/vega-embed#options for more details.
        dvc.Vega(id="altair-chart", opt={"renderer": "svg", "actions": False}),
    ]
)


@callback(Output("altair-chart", "spec"), Input("origin-dropdown", "value"))
def display_altair_chart(origin):
    source = data.cars()

    if origin != "All":
        source = source[source["Origin"] == origin]

    chart = (
        alt.Chart(source)
        .mark_circle(size=60)
        .encode(
            x="Horsepower",
            y="Miles_per_Gallon",
            color="Origin",
            # color=alt.Color("Origin").scale(domain=["Europe", "Japan", "USA"]),
            tooltip=["Name", "Origin", "Horsepower", "Miles_per_Gallon"],
        )
        .interactive()
    )
    return chart.to_dict()


if __name__ == "__main__":
    app.run(debug=True)

# Setting up Keyword Dataframe for reference in Dashboard

In [17]:
input_path = "../data_example"

keyword_df = os.path.join(input_path, "keyword_df.csv")
keyword_df = pd.read_csv(keyword_df)

keyword_df

,nctId,status,disease,intervention_type,intervention_name,durationMonths
0,NCT06475872,COMPLETED,Primary Hypercholesterolemia,DRUG,Crecheck Tablet 2.5 mg,0.566667
1,NCT06475872,COMPLETED,Combined Hyperlipidemia,DRUG,Crecheck Tablet 2.5 mg,0.566667
2,NCT06473402,COMPLETED,Muscle Tightness,DIAGNOSTIC_TEST,Muscle Strength Test,0.633333
3,NCT06473402,COMPLETED,Muscle Spasm,DIAGNOSTIC_TEST,Muscle Strength Test,0.633333
4,NCT06477575,COMPLETED,"ADHD, Deficits in Motor Control and Perception",DEVICE,neurocognitive therapy based on new technologies,0.633333
...,...,...,...,...,...,...
692,NCT06475430,NOT_YET_RECRUITING,Fecal Microbiota Transplantation,BIOLOGICAL,fecal microbiota transplantation,18.166667
693,NCT06471218,NOT_YET_RECRUITING,Hyperlipidemia,DRUG,SHR-1918; SHR-1918 Placebo,10.133333
694,NCT06474299,NOT_YET_RECRUITING,Smoking Cessation,DRUG,Varenicline,46.633333
695,NCT06474299,NOT_YET_RECRUITING,Electronic Cigarette Use,DRUG,Varenicline,46.633333


### Simply Looking thru options

In [4]:
# Testing dropwdown with keyword_df
app = Dash(
    __name__,
    # external_stylesheets=["https://codepen.io/chriddyp/pen/bWLwgP.css"]
)

app.layout = html.Div(
    [
        #  still would need to change id's then add to @callback's Input and Output lists for actual updates
        html.H1("Status"),
        dcc.Checklist(list(set(keyword_df["status"])), id="origin-dropdown"),
        html.H1("Disease"),
        dcc.Dropdown(
            list(set(keyword_df["disease"])), "All", id="origin-dropdown", multi=True
        ),
        html.H1("Intervention Type"),
        dcc.Checklist(list(set(keyword_df["intervention_type"])), id="origin-dropdown"),
        html.H1("Intervention Name"),
        dcc.Dropdown(
            list(set(keyword_df["intervention_name"])), "All", id="origin-dropdown"
        ),
        # Optionally, you can pass options to the Vega component.
        # See https://github.com/vega/vega-embed#options for more details.
        # dash_table.DataTable(keyword_df, id = 'table'),
        dash_table.DataTable(
            keyword_df.to_dict("records"),
            [{"name": i, "id": i} for i in keyword_df.columns],
            id="tbl",
        ),
        dvc.Vega(id="altair-chart", opt={"renderer": "svg", "actions": False}),
    ]
)


@callback(Output("altair-chart", "spec"), Input("origin-dropdown", "value"))
def display_altair_chart(origin):
    source = data.cars()

    if origin != "All":
        source = source[source["Origin"] == origin]

    chart = (
        alt.Chart(source)
        .mark_circle(size=60)
        .encode(
            x="Horsepower",
            y="Miles_per_Gallon",
            color="Origin",
            # color=alt.Color("Origin").scale(domain=["Europe", "Japan", "USA"]),
            tooltip=["Name", "Origin", "Horsepower", "Miles_per_Gallon"],
        )
        .interactive()
    )

    return chart.to_dict()


if __name__ == "__main__":
    app.run(debug=True)

---------------------------------------------------------------------------
DuplicateIdError                          Traceback (most recent call last)
DuplicateIdError: Duplicate component id found in the initial layout: `origin-dropdown`



### Find study descriptions for output in dashboard

In [23]:
input_path = "../data_example/description_df.csv"

description_df = pd.read_csv(input_path)

description_df

,nctId,description,durationMonths
0,NCT06475872,This study is to compare and evaluate the safe...,0.566667
1,NCT06473402,Flexibility is the ability of the muscle to mo...,0.633333
2,NCT06477575,To scientifically demonstrate the improvements...,0.633333
3,NCT06470724,Autism Spectrum Disorder (ASD) is a brain deve...,6.700000
4,NCT06470464,Thalidomide (THD) is currently widely used in ...,31.600000
...,...,...,...
420,NCT06471920,Chronic low back pain (LBP) imposes tremendous...,44.566667
421,NCT06475430,"This study is a rigorous single-center, prospe...",18.166667
422,NCT06471218,Evaluation of the efficacy and safety of multi...,10.133333
423,NCT06474299,To be efficient and maximize translation poten...,46.633333


In [24]:
description_df.loc[
    description_df.nctId == description_df["nctId"][0], "description"
].item()

'This study is to compare and evaluate the safety and pharmacokinetic characteristics of Hanlim Pharm. Co., Ltd.\'s "Crecheck Tablet 2.5 mg (Rosuvastatin Calcium)" as the test drug and AstraZeneca Korea Co., Ltd.\'s "Crestor Tablet 5 mg (Rosuvastatin Calcium)" as the reference drug in healthy adults.'

### BEST ONE SO FAR, STILL NEED WAY TO GET LIST OF LONGER UNINTUITIVE THINGS LIKE INTERVENTION TYPE TO SHOW UP AS OPTIONS PPL CAN SEE AND UPDATE THE TABLE

In [25]:
keyword_df["id"] = keyword_df["nctId"]
keyword_df.set_index("id", inplace=True, drop=False)

app = Dash(__name__)

app.layout = html.Div(
    [
        dcc.Dropdown(
            list(set(keyword_df["intervention_type"])),
            "All",
            placeholder="-Select an Intervention Type-",
            id="intervention-type-dropdown",
            multi=True,
        ),
        dcc.Checklist(list(set(keyword_df["status"])), id="status-checklist"),
        html.H1("Clinical Trials"),
        dash_table.DataTable(
            id="datatable-row-ids",
            columns=[
                {"name": i, "id": i, "deletable": True}
                for i in keyword_df.columns
                # omit the id column
                if i != "id"
            ],
            data=keyword_df.to_dict("records"),
            editable=True,
            filter_action="native",
            sort_action="native",
            sort_mode="multi",
            row_selectable="multi",
            row_deletable=True,
            selected_rows=[],
            page_action="native",
            page_current=0,
            page_size=10,
        ),
        html.H1("Study Description"),
        html.Div(
            id="datatable-row-ids-container",
            style={
                "fontSize": 24,
                #   'color': 'blue'
            },
        ),
        html.H1("Trial Duration"),
        html.Div(
            id="trial-dura-text",
            style={
                "fontSize": 24,
                #   'color': 'blue'
            },
        ),
        html.H1("Embedding Stuff"),
        # SHAP Features for Important things for this things go here
        html.H1("Visualizations"),
        # Visualization(s) Go Here
    ]
)


@callback(
    Output("datatable-row-ids-container", "children"),
    Output("trial-dura-text", "children"),
    # Input("intervention-type-dropdown", 'int_type'),
    # Input("status-checklist", 'status'),
    Input("datatable-row-ids", "derived_virtual_row_ids"),
    Input("datatable-row-ids", "selected_row_ids"),
    Input("datatable-row-ids", "active_cell"),
)
def update_graphs(row_ids, selected_row_ids, active_cell):
    # When the table is first rendered, `derived_virtual_data` and
    # `derived_virtual_selected_rows` will be `None`. This is due to an
    # idiosyncrasy in Dash (unsupplied properties are always None and Dash
    # calls the dependent callbacks when the component is first rendered).
    # So, if `rows` is `None`, then the component was just rendered
    # and its value will be the same as the component's dataframe.
    # Instead of setting `None` in here, you could also set
    # `derived_virtual_data=df.to_rows('dict')` when you initialize
    # the component.
    selected_id_set = set(selected_row_ids or [])

    if row_ids is None:
        dff = keyword_df
        # pandas Series works enough like a list for this to be OK
        row_ids = keyword_df["id"]
    else:
        dff = keyword_df.loc[row_ids]

    # setting nctId variable
    active_row_id = active_cell["row_id"] if active_cell else None

    colors = [
        (
            "#FF69B4"
            if id == active_row_id
            else "#7FDBFF" if id in selected_id_set else "#0074D9"
        )
        for id in row_ids
    ]

    description = (
        description_df.loc[description_df.nctId == active_row_id, "description"].item()
        if active_cell
        else "[Study Description Goes Here]"
    )
    dura = (
        str(
            description_df.loc[
                description_df.nctId == active_row_id, "durationMonths"
            ].item()
        )
        + " months"
        if active_cell
        else "[Trial Duration Goes Here]"
    )

    return description, dura


# [
#         dcc.Graph(
#             id=column + "--row-ids",
#             figure={
#                 "data": [
#                     {
#                         "x": dff["intervention_type"],
#                         "y": dff[column],
#                         "type": "bar",
#                         "marker": {"color": colors},
#                     }
#                 ],
#                 "layout": {
#                     "xaxis": {"automargin": True},
#                     "yaxis": {"automargin": True, "title": {"text": column}},
#                     "height": 250,
#                     "margin": {"t": 10, "l": 10, "r": 10},
#                 },
#             },
#         )
#         # check if column exists - user may have deleted it
#         # If `column.deletable=False`, then you don't
#         # need to do this check.
#         for column in ["status", "disease"]
#         if column in dff
#     ]


if __name__ == "__main__":
    app.run(debug=True)